# **1. 캐글**

[캐글(Kaggle)](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci)은 데이터 과학자와 머신러닝 엔지니어들이 학습, 협업, 경쟁할 수 있는 온라인 플랫폼입니다. 구글에 소속된 이 플랫폼은 다양한 데이터셋과 머신러닝 문제를 제공하며, 사용자들이 자신의 모델을 개발하고 성능을 경쟁적으로 평가받을 수 있는 경진대회도 열립니다. 초보자는 데이터를 다루는 실습을 하고, 경험 많은 전문가들은 포트폴리오를 확장하거나 상금을 받을 기회를 얻을 수 있습니다. 또한, 커뮤니티 포럼과 튜토리얼, 노트북 공유를 통해 지식을 공유하고 학습할 수 있는 환경을 제공합니다. 캐글은 데이터 분석과 머신러닝을 배우고 실제 문제에 적용해 보고 싶은 사람들에게 유용한 플랫폼입니다.

# **2. Online Retail**

아래 사이트에 접속해 데이터를 다운로드합니다.

In [ ]:
from google.colab import files
files.upload()  # kaggle.json 업로드 창 표시
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!/bin/bash
!kaggle datasets download mashlyn/online-retail-ii-uci

In [ ]:
import pandas as pd

retail = pd.read_csv('/content/drive/MyDrive/AI활용 멀티모달 MCP PJ 시즌1/2. 데이터 분석/data/online_retail_II.csv')
retail

'''
InvoiceNo: 주문 번호

StockCode: 상품 코드

Description: 상품 설명

Quantity: 주문 수량

InvoiceDate: 주문 날짜

Price: 상품 가격

Customer ID: 고객 아이디

Country: 고객 거주지역(국가)
'''


In [ ]:
retail.info()

In [ ]:
retail.columns

In [ ]:
retail.describe()

In [ ]:
len(retail)

In [ ]:
# 각 컬럼당 null이 몇 개 있는지 확인
retail.isnull().sum()

In [ ]:
# 각 컬럼당 null이 얼마나 있는지 비율 확인
retail.isnull().mean()
#비율로 확인

In [ ]:
# 비회원/탈퇴/휴면회원 제거
# 여러 가지 경우. 데이터 분석하는 것에는 필요가 없음. 그래서 제거!
# not null인 것만 뽑아서 덮어쓰기
retail = retail[pd.notnull(retail['Customer ID'])]
retail

In [ ]:
len(retail)

In [ ]:
# 구입 수량이 0 이하인 데이터를 확인
retail[retail['Quantity'] <= 0]

In [ ]:
# 구입 수량이 1이상인 데이터만 저장
retail = retail[retail['Quantity'] >= 1]
len(retail)

In [ ]:
# 구입 가격이 0 이하인 데이터를 확인
retail[retail['Price'] <= 0]

In [ ]:
# 구입 가격이 0보다 큰 데이터만 저장
retail = retail[retail['Price'] > 0]
len(retail)

In [ ]:
# 고객의 총 지출비용(CheckoutPrice) 파생변수 만들기
# 총 지출비용(CheckoutPrice) = 가격(Price) * 수량(Quantity)
# retail.loc[:, 'CheckoutPrice'] = retail['Price'] * retail['Quantity']
retail['CheckoutPrice'] = retail['Price'] * retail['Quantity']
retail

In [ ]:
retail.info()

In [ ]:
#날짜 계산 할 수 있도록 변환 (to_datetime이라는 메서드 사용)
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])
retail.info()

In [ ]:
# 전체 매출
total_revenue = retail['CheckoutPrice'].sum()
total_revenue

In [ ]:
# 각 나라별 구매 횟수
pd.options.display.max_rows = 20
# value_counts(): 객체의 각 고유값(유니크 값)의 개수를 계산하고, 이를 내림차순으로 정렬해 반환하는 함수
retail['Country'].value_counts()
#
# retail.groupby('Country')['Quantity'].count().sort_values(ascending=False)

In [ ]:
# 국가별 매출
rev_by_countries = retail.groupby('Country')['CheckoutPrice'].sum().sort_values()
rev_by_countries

In [ ]:
import matplotlib.pyplot as plt

# matplotlib 객체 생성
# 그래프를 그릴 축(ax)과 그림(fig)을 생성

#전체 플롯 내부에 작은 플롯을 만드는게 subplots
#도화지 환경 변경하는 객체 만들어주는 코드
# 전체 배경 설정하는 애.그래프 설정하는 애 =
fig, ax = plt.subplots(figsize=(20, 10))

# 막대 그래프 그리기
#인덱스 - 나라
ax.bar(rev_by_countries.index, rev_by_countries.values)

# 라벨 설정
ax.set_xlabel('Country', fontsize=12)
ax.set_ylabel('Revenue', fontsize=12)
ax.set_title('Revenue By Country', fontsize=15)

# x축 눈금 라벨 회전
ax.set_xticks(range(len(rev_by_countries.index)))
ax.set_xticklabels(rev_by_countries.index, rotation=45)

# 그래프 출력
# 래프와 텍스트가 잘리지 않고 보기 좋게 배치되도록 자동으로 여백을 조정하는 유용한 함수
plt.tight_layout()
plt.show()

In [ ]:
rev_by_countries / total_revenue

In [ ]:
# 월별 매출 구하기
retail['InvoiceDate'].sort_values(ascending=False)

In [ ]:
# 2009-12-01 07:45:00
def extract_month(date):
    month = str(date.month) # 12 ... 3
    if date.month < 10:
        month = '0' + month #03 ....20091201 yes 2009121 no!
    return str(date.year) + month #200912 ... 200903 이런 식으로 붙을 것

In [ ]:
rev_by_month = retail.set_index('InvoiceDate').groupby(extract_month)['CheckoutPrice'].sum()
rev_by_month
#각각의 월별 매출을 전부 구한 것

In [ ]:
def plot_bar(df, xlabel, ylabel, title, figsize=(20, 10), fontsize=12, titlesize=15, rotation=45):
    fig, ax = plt.subplots(figsize=figsize)
    ax.bar(df.index, df.values)
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    ax.set_title(title, fontsize=titlesize)
    ax.set_xticks(df.index)
    ax.set_xticklabels(df.index, rotation=rotation, fontsize=fontsize)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_bar(rev_by_month, 'Month', 'Revenue', 'Revenue By Month')

In [ ]:
# 요일별 매출 구하기
def extract_dow(date):
    return date.dayofweek

In [ ]:
rev_by_dow = retail.set_index('InvoiceDate').groupby(lambda date: date.dayofweek)['CheckoutPrice'].sum()
rev_by_dow

In [ ]:
import numpy as np

DAY_OF_WEEK = np.array(['Mon', 'Tue', 'Web', 'Thur', 'Fri', 'Sat', 'Sun'])
rev_by_dow.index = DAY_OF_WEEK[rev_by_dow.index]
rev_by_dow.index

In [ ]:
rev_by_dow

In [ ]:
plot_bar(rev_by_dow, 'DOW', 'Revenue', 'Revenue By DOW')

In [ ]:
# 시간대별 매출 구하기
rev_by_hour = retail.set_index('InvoiceDate').groupby(lambda date: date.hour)['CheckoutPrice'].sum()
rev_by_hour

In [ ]:
plot_bar(rev_by_hour, 'Hour', 'Revenue', 'Revenu By Hour')

In [ ]:
# 요일별 매출 구하기
def extract_dow(date):
    return date.dayofweek

In [ ]:
rev_by_dow = retail.set_index('InvoiceDate').groupby(lambda date: date.dayofweek)['CheckoutPrice'].sum()
rev_by_dow

In [ ]:
import numpy as np

DAY_OF_WEEK = np.array(['Mon', 'Tue', 'Web', 'Thur', 'Fri', 'Sat', 'Sun'])
rev_by_dow.index = DAY_OF_WEEK[rev_by_dow.index]
rev_by_dow.index

In [ ]:
rev_by_dow


In [ ]:
plot_bar(rev_by_dow, 'DOW', 'Revenue', 'Revenue By DOW')


In [ ]:
# 시간대별 매출 구하기
rev_by_hour = retail.set_index('InvoiceDate').groupby(lambda date: date.hour)['CheckoutPrice'].sum()
rev_by_hour

In [ ]:
plot_bar(rev_by_hour, 'Hour', 'Revenue', 'Revenu By Hour')